In [29]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import collections
import matplotlib as plt

In [30]:
profiles = pd.read_csv('profiles.csv')

print(profiles.head())
profiles.info()

   age       body_type               diet    drinks      drugs  \
0   22  a little extra  strictly anything  socially      never   
1   35         average       mostly other     often  sometimes   
2   38            thin           anything  socially        NaN   
3   23            thin         vegetarian  socially        NaN   
4   29        athletic                NaN  socially      never   

                           education  \
0      working on college/university   
1              working on space camp   
2     graduated from masters program   
3      working on college/university   
4  graduated from college/university   

                                              essay0  \
0  about me:<br />\n<br />\ni would love to think...   
1  i am a chef: this is what that means.<br />\n1...   
2  i'm not ashamed of much, but writing public te...   
3          i work in a library and go to school. . .   
4  hey how's it going? currently vague on the pro...   

                         

In [31]:
print(profiles['education'].value_counts())

education
graduated from college/university    23959
graduated from masters program        8961
working on college/university         5712
working on masters program            1683
graduated from two-year college       1531
graduated from high school            1428
graduated from ph.d program           1272
graduated from law school             1122
working on two-year college           1074
dropped out of college/university      995
working on ph.d program                983
college/university                     801
graduated from space camp              657
dropped out of space camp              523
graduated from med school              446
working on space camp                  445
working on law school                  269
two-year college                       222
working on med school                  212
dropped out of two-year college        191
dropped out of masters program         140
masters program                        136
dropped out of ph.d program            127
d

From these counts, it is clear that OKCupid has a large user base within graduated university students. This includes a significant number of people that continued their education and graduated from a masters degree program. This suggests a possible connection between education level and usage of OKCupid

In [33]:
#Gathering all profiles 'education' details
education = profiles['education']
print(education)

0            working on college/university
1                    working on space camp
2           graduated from masters program
3            working on college/university
4        graduated from college/university
                       ...                
59941    graduated from college/university
59942        working on college/university
59943       graduated from masters program
59944        working on college/university
59945       graduated from masters program
Name: education, Length: 59946, dtype: object


In [26]:
#drugs = profiles['drugs'].replace({'never':0,'nan':0,'sometimes':1,'often':2}, inplace = True)
#profiles.reset_index()


None


0        0.0
1        1.0
2        NaN
3        NaN
4        0.0
        ... 
59941    0.0
59942    1.0
59943    0.0
59944    2.0
59945    NaN
Name: drugs, Length: 59946, dtype: float64


KeyError: ('graduated from college/university', 'graduated from high school', 'graduated from law school', 'graduated from masters program', 'graduated from med school', 'graduated from ph.d program', 'graduated from space camp', 'graduated from two-year college', 'high school', 'law school', 'masters program', 'med school', 'ph.d program', 'space camp', 'two-year college', 'working on college/university', 'working on high school', 'working on law school', 'working on masters program', 'working on med school', 'working on ph.d program', 'working on space camp', 'working on two-year college')